<a href="https://colab.research.google.com/github/AlexandreGuerra-prod/_ENAP/blob/main/fluxograma_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Importações e Setup
import pytesseract  # Biblioteca para OCR (Reconhecimento Óptico de Caracteres)
from PIL import Image, ImageDraw  # Biblioteca para manipulação de imagens
import matplotlib.pyplot as plt  # Biblioteca para criação de gráficos
import torch  # Biblioteca para operações de Tensor, necessária para modelos de ML
from transformers import BlipProcessor, BlipForConditionalGeneration  # Para processamento e geração de legendas
import networkx as nx  # Biblioteca para manipulação de grafos
import os  # Biblioteca para interações com o sistema operacional
from fpdf import FPDF  # Biblioteca para gerar arquivos PDF

# 2. Função para Extração de Texto com OCR - Pytesseract
def extract_text_blocks(image_path):
    """
    Esta função extrai blocos de texto de uma imagem utilizando OCR e retorna suas coordenadas.

    :param image_path: Caminho para a imagem de entrada.
    :return: Lista de blocos de texto com informações de coordenadas.
    """
    image = Image.open(image_path).convert("RGB")  # Abre a imagem e a converte para RGB
    data = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)  # Realiza OCR
    draw = ImageDraw.Draw(image)  # Cria um objeto para desenhar na imagem
    blocks = []  # Lista para armazenar os blocos de texto

    # Percorre cada item retornado pelo OCR
    for i in range(len(data['text'])):
        text = data['text'][i].strip()  # Remove espaços em branco
        if text:  # Verifica se o texto não está vazio
            x, y, w, h = data['left'][i], data['top'][i], data['width'][i], data['height'][i]
            blocks.append({
                'text': text,  # Texto extraído
                'bbox': (x, y, w, h)  # Caixa delimitadora: posição e tamanho
            })
            # Desenha um retângulo na imagem ao redor do texto
            draw.rectangle([x, y, x + w, y + h], outline='red', width=2)
            draw.text((x, y - 10), text, fill='blue')  # Adiciona o texto acima do retângulo

    # Exibe a imagem com os blocos de texto destacados
    plt.figure(figsize=(12, 8))
    plt.imshow(image)
    plt.axis('off')  # Remove os eixos
    plt.title("Blocos de Texto Detectados")
    plt.show()

    return blocks  # Retorna a lista de blocos de texto


# 3. Análise com BLIP (Modelo Multimodal de Imagem)
def analyze_with_blip(image_path):
    """
    Esta função gera uma descrição para a imagem usando um modelo de linguagem e visão (BLIP).

    :param image_path: Caminho para a imagem de entrada.
    :return: Descrição da imagem gerada pelo modelo.
    """
    # Inicializa o processador e o modelo BLIP
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    image = Image.open(image_path).convert("RGB")  # Abre a imagem
    inputs = processor(images=image, return_tensors="pt")  # Processa a imagem para tensor
    out = model.generate(**inputs)  # Gera a descrição
    caption = processor.decode(out[0], skip_special_tokens=True)  # Decodifica a saída
    return caption  # Retorna a descrição


# 4. Construção e Análise de Grafo
def draw_and_identify_bottlenecks(blocks):
    """
    Esta função cria um grafo a partir dos blocos de texto e identifica gargalos.

    :param blocks: Lista de blocos de texto.
    :return: Tupla (grafo, lista de gargalos).
    """
    G = nx.DiGraph()  # Cria um grafo direcionado
    for i, block in enumerate(blocks):
        G.add_node(i, label=block['text'])  # Adiciona nó para cada bloco de texto
        if i > 0:
            G.add_edge(i - 1, i)  # Cria uma ligação entre blocos sequenciais

    pos = nx.spring_layout(G)  # Calcula a posição dos nós no grafo
    labels = nx.get_node_attributes(G, 'label')  # Obtém rótulos para os nós
    plt.figure(figsize=(10, 6))
    nx.draw(G, pos, with_labels=True, labels=labels, node_size=2000, node_color='lightblue', font_size=10)

    # Identifica gargalos: nós com alto grau de entrada
    bottlenecks = [n for n in G.nodes if G.in_degree(n) > 1]  # Nós que recebem entrada de múltiplos nós
    nx.draw_networkx_nodes(G, pos, nodelist=bottlenecks, node_color='red')  # Destaca os gargalos em vermelho
    plt.title("Grafo Gerado e Gargalos (vermelho)")
    plt.axis('off')  # Remove os eixos
    plt.savefig("grafo_corrigido.png")  # Salva a imagem do grafo
    plt.show()
    return G, bottlenecks  # Retorna o grafo e os gargalos


# 5. Sugestões para Correção de Gargalos
def sugerir_correcao(grafo, gargalos):
    """
    Esta função gera sugestões para melhorar os gargalos detectados no grafo.

    :param grafo: O grafo analisado.
    :param gargalos: Lista de nós que são gargalos.
    :return: Lista de sugestões.
    """
    sugestoes = []  # Lista para armazenar sugestões
    for g in gargalos:
        label = grafo.nodes[g].get('label', f'Nó {g}')  # Obtém a etiqueta do nó, se disponível
        sugestao = f"O bloco '{label}' é um gargalo. Sugestão: divida o fluxo de entrada ou revise a lógica de dependência."
        sugestoes.append(sugestao)  # Adiciona a sugestão à lista
    return sugestoes  # Retorna a lista de sugestões


# 6. Gerar PDF da Imagem Corrigida
def gerar_pdf(imagem_path, sugestoes):
    """
    Esta função gera um PDF com a imagem do fluxograma e as sugestões de correção.

    :param imagem_path: Caminho para a imagem do grafo.
    :param sugestoes: Lista de sugestões para correção.
    """
    pdf = FPDF()  # Cria uma instância do PDF
    pdf.add_page()  # Adiciona uma página ao PDF
    pdf.set_font("Arial", size=12)  # Define a fonte e tamanho
    pdf.cell(200, 10, txt="Fluxograma Corrigido", ln=True, align='C')  # Título do PDF
    pdf.image(imagem_path, x=10, y=30, w=190)  # Adiciona a imagem do grafo ao PDF
    pdf.ln(120)  # Espaçamento
    pdf.set_font("Arial", size=10)  # Define a fonte para sugestões
    pdf.cell(200, 10, txt="Sugestões de Correção:", ln=True)  # Título das sugestões
    for s in sugestoes:
        pdf.multi_cell(0, 8, txt="- " + s)  # Adiciona cada sugestão como uma nova linha
    pdf.output("fluxograma_corrigido.pdf")  # Salva o PDF com o nome especificado


# 7. Execução Exemplo do Pipeline Completo
# Certifica-se de que o diretório 'imagens' existe
if not os.path.exists("imagens"):
    os.makedirs("imagens")  # Cria o diretório se não existir
    print("Diretório 'imagens' criado.")

# URL RAW da imagem no seu repositório GitHub
image_url = "https://raw.githubusercontent.com/AlexandreGuerra-prod/_ENAP/main/exemplo_fluxograma.png"
local_image_path = "imagens/exemplo_fluxograma.png"  # Caminho onde a imagem será salva localmente

# Baixa a imagem do GitHub para o caminho local usando wget
!wget -O {local_image_path} {image_url}  # O -O especifica o nome do arquivo de saída local

print(f"Imagem baixada para: {local_image_path}")

# Execução das funções
image_path = local_image_path  # Usa o caminho local da imagem

# 7. Executando as etapas do pipeline
blocks = extract_text_blocks(image_path)  # Extração dos blocos de texto
caption = analyze_with_blip(image_path)  # Análise multimodal com BLIP
print("Descrição:", caption)  # Exibe a descrição gerada pela BLIP
G, bottlenecks = draw_and_identify_bottlenecks(blocks)  # Geração do grafo e identificação de gargalos
sugestoes = sugerir_correcao(G, bottlenecks)  # Sugestões para correções
gerar_pdf("grafo_corrigido.png", sugestoes)  # Gera o PDF com as sugestões

# Mensagens de log para o usuário
print("\n🔍 Executando OCR e extração de blocos:")
blocks = extract_text_blocks(image_path)

print("\n🧠 Análise multimodal com BLIP:")
caption = analyze_with_blip(image_path)
print("\nDescrição do Diagrama:", caption)

print("\n🔗 Construção do grafo e identificação de gargalos:")
G, bottlenecks = draw_and_identify_bottlenecks(blocks)
print("\nGargalos detectados nos nós:", bottlenecks)

print("\n🛠 Sugestões de melhoria:")
sugestoes = sugerir_correcao(G, bottlenecks)
for s in sugestoes:
    print("-", s)

print("\n📄 Exportando grafo corrigido para PDF...")
gerar_pdf("grafo_corrigido.png", sugestoes)
print("PDF salvo como 'fluxograma_corrigido.pdf'")